# LECTURE 10/09/2025


### Algorithms
An algorithm is a mathematical definition where computation is seen as a sequence of steps constructing the final result.

### Fondumental formalisms:
- General recursive functions
- lambda calculus
- rewriting systems
- formal semantics

### Declarative programming (imperative): functional, logic, constraint based etc...
Declarative programming is not efficient due to the fact that it cannot use ephemeral data strucuter like arrays.

Persistent data structures address this issue:
- their interfaces only exposes operations that do not modify the structure in place but instead return new modified structures
- their implementations achieves the complexity of the best known ephemeral structures

In  imperative programming and classical algorithms these data structures can be relevant:
- backtracking are trivial
- complete history of updates of the structure
- no in-place modification allows memory sharing between successive versions
- concurrency


A stack is a persistent data structure with the following operations:
- init
- push(v)
- top
- pop

Persistent stack implemented as a linked list:

```

class Stack{
    private int hd; 
    private Stack tl;
    static Stack empty = null;
    static Stack push(int v, Stack s){
        Stack t = new Stack; t.hd = v; t.tl = s; return t;
    }

    static int top(Stack s){return s.hd};
    static Stack pop(Stack s){return s.tl};
}


```

- Immutability: operations do not overwrite state, they return a new value and it is not possible to mutate a shared node afterwards
- Structural sharing: new versions reuse unchanged parts
- Persistence: old versions remain accessible
- Safe and efficient persistence: immutability and structural sharing OR fat node fersioning with bounded histories

If immutability is missing then a write can mutate a node that newer versions still point to and the old snapshots will get corrupted.
If there is no safe sharing then the updates will copy the whole structure (O(n)) resulting in poor time/space.

## Dobkin and Lipton’s algorithm

**Preprocessing** of $n$ line segments involves creating a grid to quickly locate any point.

* First, we sort all the x-coordinates of the segment endpoints. These vertical lines create $O(n)$ "slabs" or vertical strips.
* Within each slab, the segments don't cross, so their vertical order is fixed. We create a sorted list of all segments for each slab.

This preparation takes $O(n^2)$ space because each of the $O(n)$ slabs might contain a list of all $O(n)$ segments.

**Querying** for a point $P = (x, y)$:

1.  We use binary search on the x-coordinates to find which slab contains $P$. This takes $O(\log n)$ time.
2.  Within that slab's sorted list, we use another binary search to find the two segments immediately above and below $P$. This also takes $O(\log n)$ time.

The total query time is a very fast **$O(\log n)$**, but the initial space and time cost is high.



### Reducing space and preprocessing time

The key insight is that adjacent slabs are nearly identical. When moving from one slab to the next, only a few segments begin or end, while the rest maintain their relative vertical order. Storing a separate, full list for each slab is incredibly redundant.

The solution is to use a **persistent data structure**. Instead of a simple array, we'll store each slab's segment list in a persistent balanced binary search tree. This allows adjacent slabs to share most of their structure, drastically reducing memory.

---

## Persistent insertion in a BST

A persistent BST never modifies its nodes. Instead, an update returns a new tree with the change, leaving the original intact. This is achieved through **path copying**.

1.  **Search**: To insert a new segment `g`, we first search the tree for its correct position, just like a normal BST.
2.  **Copy Path**: As we traverse from the root to the insertion point, we create a copy of each node along the path. These new nodes point to the original, untouched subtrees that were not on the path. This is called **structural sharing**.
3.  **Insert**: At the end of the new path, we add the new node for `g`.
4.  **Rebalance**: If the tree becomes unbalanced, we perform rotations on the *newly copied path*, again without altering the original tree.

This operation creates a new root, which represents the updated tree. The time and space cost for a single insertion is only **$O(\log n)$**.



## Improved point location algorithm

This approach, known as the **sweep-line algorithm**, refines the slab method by using a persistent BST.

**Preprocessing:**

1.  We process the segment endpoints in order from left to right, as if a vertical line is "sweeping" across the plane.
2.  When the sweep-line encounters the start of a segment, we **persistently insert** it into a balanced BST, which keeps the segments sorted by their vertical position. When it encounters the end of a segment, we persistently delete it.
3.  We store a reference to the root of the BST at each endpoint. Each of these stored roots represents the state of the segments within a slab.

This preprocessing takes **$O(n \log n)$** time and space because each of the $2n$ endpoints requires one persistent tree operation, costing $O(\log n)$ each.

**Querying** for a point $P = (x, y)$:

1.  We first use binary search on the sorted list of x-coordinates (our "slabs") to find the correct version of the BST.
2.  Then, we search within that specific BST to find the segments above and below $P$.

The total query time remains an optimal **$O(\log n)$**.

---

## The Sarnak-Tarjan algorithm

Sarnak and Tarjan further optimized the space from $O(n \log n)$ down to a linear **$O(n)$** by using a different implementation of persistent BSTs known as the "**fat nodes**" technique.

Instead of copying nodes on every update, this method modifies nodes in-place but keeps a history of the changes. Each field in a node (like `left`, `right`, or `color`) is replaced with a list of its historical values, each tagged with a version number or "time."

### Example of BST with fat nodes

Imagine a node's `left` child pointer. In a normal BST, it holds one value. In a fat node, it holds a history:

* `left`: `[(version 1, NodeA), (version 5, NodeB), (version 12, NodeC)]`

To find the `left` child at version 10, you search this list to find the latest entry with a version less than or equal to 10 (which would be `NodeB`).

### Memory usage

With fat nodes, each field modification adds one entry to a history list, costing $O(1)$ space. After $n$ insertions and $n$ deletions in a balanced BST, which require an amortized $O(1)$ updates each, the total space used is only **$O(n)$**. This is a significant improvement over the $O(n \log n)$ space required by the path-copying method.

### Access time in the structure

A naive implementation of fat nodes would be slow. Accessing a field's value at a specific time would require a search through its modification history, taking $O(\log k)$ time, where $k$ is the number of modifications. This could slow down the total BST search time to $O(\log^2 n)$.

Sarnak and Tarjan's crucial optimization is a **hybrid approach**: they allow a node's history list to grow only to a certain size. When a history list becomes full, the entire fat node is copied, and its history is consolidated. This bounds the history search time, bringing the overall query time back to an optimal **$O(\log n)$**.

The result is the best of both worlds: a data structure for planar point location with **$O(n)$** space and **$O(\log n)$** query time.

# LECTURE 24/09/2025

## Lambda calculus
The lambda calculus is a model of computation, it only has one type of value (functions) and it focuses on variable binding and sustition as computation.

Studying λ-calculus provides a clear foundation for understanding first-class functions, lexical scoping, and programming language semantics (operational, denotational, and type-theoretic). It has significantly influenced languages such as Lisp, ML, Haskell, and JavaScript, and most programming language calculi can be seen as extensions of λ-calculus.

### Syntax of λ-calculus

e ::= x | λx.e | ee

- Variables: x,y,z,f,g,...
- Abstraction: λx.e
- Application: ${e_1}{e_2}$
- Parentheses associate left: a b c = (a b) c

### Reading λ-calculus in JS

| λ-calculus                  | JS                         |
|-----------------------------|----------------------------|
| λx.(2+x)                    | x => 2 + x                 |
| (λx.(2+x))5                 | (x => 2 + x)(5)            |
| (λf.(f3))(λx.(x+1))         | (f => f(3))(x => x + 1)    |

---

### Function composition example  
Build f o f and apply to x  

| JS                                      | λ-calculus                            |
|-----------------------------------------|---------------------------------------|
| f => (x => f(f(x)))                     | λf.λx.f(x)                            |
| ((f => (x => f(f(x))))(x => x+1))(4)    | ((λf.λx.f(fx))(λx.x+1))4              |

---

### Free and bound variables  

| Free vars                               | Bound vars                            |
|-----------------------------------------|---------------------------------------|
| FV(x)={x}                               | BV(x) = 0                             |
| FV(λx.e)=FV(e)\{x}                      | BV(λx.e)=$BV(e)\cup{x}$               |
| FV(${e_1}{e_2}$)=${FV(e_1)}\cup{FV(e_2)}$ | BV(${e_1}{e_2}$)=${BV(e_1)}\cup{BV(e_2)}$ |


### Substitution
$e_1[x:=e_2]$ replace every free x in $e_1$ by $e_2$

### Evaluation (β-reduction):
$(λx . e_1) e_2 → e_1 [x := e_2]$

### Capture-avoiding substitution

$x [x := e] = e$

$y [x := e] = y if y!= x$

$({e_1}{e_2}) [x := e] = ({e_1} [x := e]) ({e_2} [x := e])$

$(λy . e_1) [x := e] = λy . e_1 [x := e]$


problems solved:

1. 
$(λx.2 + x) 5$

lambda reduction: $(2+x)[x:=5]$

substitution (2+x): $(2 [x := 5]) + (x [x := 5]) [because x is substituted by [x:=5]]$

$(2[x:=5])$ becomes 2

$(x[x:=5])$ given the rule $x [x := e] = e$

2. 
$(λf . f 3) (λx . x + 1)$

(λparameter . body) argument

The rule for β-reduction says that this simplifies to one action:

body [parameter := argument]

$f3 (f:=(λx . x + 1))$

$((λx . x + 1) 3)$

$(x+1)[x:=3]$

$3+1=4$

---